In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset

In [3]:
from torch.utils import data as d

In [4]:
import collections

In [5]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,f1_score,classification_report,matthews_corrcoef,accuracy_score,confusion_matrix

In [6]:
class NN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.out(output)
        return output

In [7]:
with open("X.pkl","rb") as file:
    X = pickle.load(file)
with open("Y.pkl","rb") as file:
    Y = pickle.load(file)    

In [212]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [213]:
X=np.asarray(X_train).reshape(len(X_train),-1)

In [214]:
X_test = np.asarray(X_test).reshape(len(X_test),-1)

In [215]:
y=np.asarray(y_train).reshape(len(X_train))

In [216]:
y_test = np.asarray(y_test).reshape(X_test.shape[0]) 

In [43]:
from sklearn.ensemble import RandomForestClassifier

In [156]:
clf = RandomForestClassifier(max_depth=400, random_state=0,n_estimators=250,n_jobs=4,verbose=1,max_features=400)
clf.fit(X, y)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  5.0min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 27.3min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed: 34.0min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=400, max_features=400, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=4,
                       oob_score=False, random_state=0, verbose=1,
                       warm_start=False)

In [157]:
y_pred = clf.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    0.2s finished


In [165]:
collections.Counter(y_pred)

Counter({1: 6418, 0: 1393})

In [161]:
f1_score(y_test, y_pred)

0.8573333333333333

In [162]:
matthews_corrcoef(y_test, y_pred)

0.41287038082100097

In [163]:
accuracy_score(y_test, y_pred)

0.7808219178082192

In [164]:
roc_auc_score(y_test, y_pred)

0.6749883743153786

In [152]:
target_names = ['Unstable', 'Stable']

In [166]:
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    Unstable       0.69      0.43      0.53      2229
      Stable       0.80      0.92      0.86      5582

    accuracy                           0.78      7811
   macro avg       0.74      0.67      0.69      7811
weighted avg       0.77      0.78      0.76      7811



In [26]:
with open("X.pkl","rb") as file:
    X = pickle.load(file)
with open("Y.pkl","rb") as file:
    Y = pickle.load(file)    

In [8]:
indices = []
count = 0
while count < len(Y)-sum(Y):
    index = np.random.randint(len(Y))
    if Y[index] == 1 and index not in indices:
        indices.append(index)
        count=count+1
    

In [9]:
final_X = []
final_Y = []
for i in indices:
    final_X.append(X[i])
    final_Y.append(Y[i])

In [10]:
for i in range(len(X)):
    if Y[i] == 0:
        final_X.append(X[i])
        final_Y.append(Y[i])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(final_X, final_Y, test_size=0.2, random_state=42)

In [12]:
X_train = torch.Tensor(X_train).permute(0,2,1)

In [13]:
y_train = torch.Tensor(y_train)

In [14]:
X_test = torch.Tensor(X_test).permute(0,2,1)

In [15]:
y_test = torch.Tensor(y_test)

In [16]:
train_set = d.TensorDataset(X_train,y_train)

In [17]:
test_set = d.TensorDataset(X_test,y_test)

In [18]:
train_loader = torch.utils.data.DataLoader(train_set,
        batch_size=16, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_set,
        batch_size=X_test.shape[0], shuffle=True)

In [19]:
net = NN(400, 32, 2)

In [20]:
import pyro
from pyro.distributions import Normal, Categorical
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [21]:
log_softmax = nn.LogSoftmax(dim=1)

In [22]:
def model(x_data, y_data):
    
    fc1w_prior = Normal(loc=torch.zeros_like(net.fc1.weight), scale=torch.ones_like(net.fc1.weight))
    fc1b_prior = Normal(loc=torch.zeros_like(net.fc1.bias), scale=torch.ones_like(net.fc1.bias))
    
    outw_prior = Normal(loc=torch.zeros_like(net.out.weight), scale=torch.ones_like(net.out.weight))
    outb_prior = Normal(loc=torch.zeros_like(net.out.bias), scale=torch.ones_like(net.out.bias))
    
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior,  'out.weight': outw_prior, 'out.bias': outb_prior}
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", net, priors)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    
    lhat = log_softmax(lifted_reg_model(x_data))
    
    pyro.sample("obs", Categorical(logits=lhat), obs=y_data)

In [23]:
softplus = torch.nn.Softplus()

def guide(x_data, y_data):
    
    # First layer weight distribution priors
    fc1w_mu = torch.randn_like(net.fc1.weight)
    fc1w_sigma = torch.randn_like(net.fc1.weight)
    fc1w_mu_param = pyro.param("fc1w_mu", fc1w_mu)
    fc1w_sigma_param = softplus(pyro.param("fc1w_sigma", fc1w_sigma))
    fc1w_prior = Normal(loc=fc1w_mu_param, scale=fc1w_sigma_param)
    # First layer bias distribution priors
    fc1b_mu = torch.randn_like(net.fc1.bias)
    fc1b_sigma = torch.randn_like(net.fc1.bias)
    fc1b_mu_param = pyro.param("fc1b_mu", fc1b_mu)
    fc1b_sigma_param = softplus(pyro.param("fc1b_sigma", fc1b_sigma))
    fc1b_prior = Normal(loc=fc1b_mu_param, scale=fc1b_sigma_param)
    # Output layer weight distribution priors
    outw_mu = torch.randn_like(net.out.weight)
    outw_sigma = torch.randn_like(net.out.weight)
    outw_mu_param = pyro.param("outw_mu", outw_mu)
    outw_sigma_param = softplus(pyro.param("outw_sigma", outw_sigma))
    outw_prior = Normal(loc=outw_mu_param, scale=outw_sigma_param).independent(1)
    outb_mu = torch.randn_like(net.out.bias)
    outb_sigma = torch.randn_like(net.out.bias)
    outb_mu_param = pyro.param("outb_mu", outb_mu)
    outb_sigma_param = softplus(pyro.param("outb_sigma", outb_sigma))
    outb_prior = Normal(loc=outb_mu_param, scale=outb_sigma_param)
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior, 'out.weight': outw_prior, 'out.bias': outb_prior}
    
    lifted_module = pyro.random_module("module", net, priors)
    
    return lifted_module()

In [24]:
optim = Adam({"lr": 0.001})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [35]:
epochs = 20
loss = 0

for j in range(epochs):
    loss = 0
    for batch_id, data in enumerate(train_loader):
        # calculate the loss and take a gradient step
        loss += svi.step(data[0].view(-1,400), data[1])
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = loss / normalizer_train
    
    print("Epoch ", j, " Loss ", total_epoch_loss_train)

Epoch  0  Loss  6.327790191383539
Epoch  1  Loss  6.508235937392257


KeyboardInterrupt: 

In [26]:
import collections

In [57]:
num_samples = 100
def predict(x):
    sampled_models = [guide(None, None) for _ in range(num_samples)]
    yhats = [model(x).data for model in sampled_models]
    mean = torch.mean(torch.stack(yhats), 0)
    return np.argmax(mean.numpy(), axis=1)

print('Prediction when network is forced to predict')
correct = 0
total = 0
for j, data in enumerate(test_loader):
    X, labels = data
    predicted = torch.tensor(predict(X.view(-1,400)))
    print("mcc- ")
    print(matthews_corrcoef(list(predicted),list(labels)))
    print("confusion_matrix- ")
    print(confusion_matrix(list(predicted),list(labels)))
    print("-------------")
    print("f1 score- ")
    print(f1_score(list(predicted),list(labels)))
    print("-------------")
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print("accuracy: %d %%" % (100 * correct / total))

Prediction when network is forced to predict
mcc- 
0.06444294959902451
confusion_matrix- 
[[ 242  168]
 [1945 2119]]
-------------
f1 score- 
0.6672964887419304
-------------
accuracy: 52 %


In [28]:
classes = ('0', '1')

In [49]:
num_samples = 100
def give_uncertainities(x):
    sampled_models = [guide(None, None) for _ in range(num_samples)]
    yhats = [F.log_softmax(model(x.view(-1,400)).data, 1).detach().numpy() for model in sampled_models]
    return np.asarray(yhats)

In [53]:
def test_batch(embeddings, labels, plot=True):
    predictions=[]
    y = give_uncertainities(embeddings)
    predicted_for_embeddings = 0
    correct_predictions=0
    correct_0=0
    incorrect_0=0
    correct_1=0
    incorrect_1=0

    for i in range(len(labels)):

        all_prob = []
    
        hilighted_something = False
    
        for j in range(len(classes)):
        
            highlight=False
        
            histo = []
            histo_exp = []
        
            for z in range(y.shape[0]):
                histo.append(y[z][i][j])
                histo_exp.append(np.exp(y[z][i][j]))
            
            prob = np.percentile(histo_exp, 50)
            if(prob>0.65):
                highlight = True
        
            all_prob.append(prob)
            
            if(highlight):
                hilighted_something = True
        predicted = np.argmax(all_prob)
        predictions.append(predicted)
    
        if(hilighted_something):
            predicted_for_embeddings+=1
            if(labels[i].item()==predicted):
                if predicted == 0:
                    correct_0 = correct_0 + 1
                else:
                    correct_1 = correct_1 + 1
                correct_predictions +=1.0
            else:
                if predicted == 0:
                    incorrect_0 = incorrect_0 + 1
                else:
                    incorrect_1 = incorrect_1 + 1
        else:
            if(plot):
                print("Undecided.")
    
    tn = correct_0
    fn = incorrect_0
    tp = correct_1
    fp = incorrect_1
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = 2*((precision*recall)/(precision+recall))
    mcc = ((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    print("Summary")
    print("Total embeddings: ",len(labels))
    print("Predicted for: ",predicted_for_embeddings)
    print("Accuracy when predicted: ",(correct_predictions/predicted_for_embeddings)*100)
    print("mcc - ",mcc)
    print("f1 score - ",f1_score)
    return len(labels), correct_predictions, predicted_for_embeddings,predictions,labels     

In [59]:
print('Bayesian Prediction')
correct = 0
total = 0
total_predicted_for = 0
for j, data in enumerate(test_loader):
    embeddings, labels = data
    test_batch(embeddings, labels, plot=False)

Bayesian Prediction
Summary
Total embeddings:  4474
Predicted for:  1518
Accuracy when predicted:  61.462450592885375
mcc -  0.17091668166579344
f1 score -  0.7122479094933595
